In [1]:
import urllib.request, json 
import pandas as pd
import os
import functools
import time
import sys
from env import *

In [2]:
pd.set_option('display.max_columns', None)

The following function is taken from https://realpython.com/primer-on-python-decorators/ and is used to measure the runtime of functions.

In [3]:
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

Loading sessions and dividing them to FG and H group.

In [4]:
df = pd.read_excel(PAIRED_SESSIONS, sheet_name="Sessions")
df = df.dropna(axis = 0, subset="Session")
fg_from_excel = df.loc[df['Task'] == 1]["Session"]
h_from_excel = df.loc[df['Task'] == 2]["Session"]
len(fg_from_excel) + len(h_from_excel) # should be 165

165

Loading JSONs with pairing of pageviews to Limesurvey questions.

In [5]:
with open(FG_FILES + "Sessions pairing.json") as loadfile:
    pairing_fg = json.load(loadfile)
    
with open(H_FILES + "Sessions pairing.json") as loadfile:
    pairing_h = json.load(loadfile)

Function for extracting important data from sessions JSON. For every pageview of session, *move, scroll, click and input* information are extracted and saved into dictionary. Function also pairs every pageview to Limesurvey page based on provided pairing map. Function returns dictionary containing all importants events for all pageviews of session.

In [6]:
def get_events(path_to_files, session, pairing):
    with open(path_to_files + session + ".json") as loadfile:
        data = json.load(loadfile)
        
    events = [[] for x in range(len(data['pageviews']))]
    
    aggregation_index = 0

    for pageview_count, pageview in enumerate(data['pageviews']):
      info = [
            pageview["screenWidth"],
            pageview["screenHeight"],
            pageview["width"],
            pageview["height"],
            pageview["duration"],
            pageview["inactivity"],
            pageview["startedAt"],
            pageview["clientStartedAt"]
      ]
      events[pageview_count].append(info)
        
      for baked_count, baked in enumerate(pageview['baked']):

        if baked['type'] == 'move':
          for position_count, position in enumerate(baked['args']['position']):
            events[pageview_count].append([pageview_count, pairing[pageview['id']], 'move', [position_count, position, baked['args']['duration'], baked['at']]])

        if baked['type'] == 'scroll':
          maxscroll = baked['args']['maxScroll']
          events[pageview_count].append([pageview_count, pairing[pageview['id']], 'scroll', [maxscroll, baked['at']]])

        if baked['type'] == 'click':
          try:
              text = baked['args']['text']
          except:
              text = -1
          position = baked['args']['position']
          try:
              target = baked['args']['target']
          except:
              target = -1
          at = baked['at']
          events[pageview_count].append([pageview_count, pairing[pageview['id']], 'click', [text, position, target, at]])    

        if baked['type'] == 'input' and baked['args']['type'] == 'textarea':
          value = baked['args']['value']
          try:
              position = baked['args']['position']
          except:
              position = -1
          try:
              target = baked['args']['target']
          except:
              targer = -1
          changes = baked['args']['changes']
          duration = baked['args']['duration']
          at = baked['at']
          events[pageview_count].append([pageview_count, pairing[pageview['id']], 'input', [value, position, target, changes, duration, at]])
            
    return events

Function for creating dataframe containing all important events. Function calls get_events for every session from provided list. For every session function creates dataframe row, where columns are marked by Limesurvey page numbers. To this row, function inserts data on correct places based on pairing. Function then returns created dataframe.

In [7]:
@timer
def create_df(path, sessions, pairing):
    df = pd.DataFrame()
    df['Session'] = []
    for x in range(68):
        df['Page_{}'.format(x)] = []
    
    for session in sessions:
        # print(session)
        events = get_events(path, session, pairing[session]) # list of actions where key is pageview, not page
        row = {'Page_{}'.format(x):[] for x in range(68)} # line representing pages, 1 page can consists of multiple pageviews, because of enabled returning
        row['Session'] = session
        for pageview_events in events:
            if len(pageview_events) > 1 and pageview_events[1][1] != 'unknown':
                row['Page_{}'.format(pageview_events[1][1])].append(pageview_events)
        df = df.append(row, ignore_index=True)
        
    return df

Creating dataframe for FG sessions.

In [8]:
df_fg = create_df(FG_FILES, fg_from_excel, pairing_fg)
df_fg[:5]

C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\117512531.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\117512531.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\117512531.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\117512531.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users

Finished 'create_df' in 10.3777 secs


,Session,Page_0,Page_1,Page_2,Page_3,Page_4,Page_5,Page_6,Page_7,Page_8,Page_9,Page_10,Page_11,Page_12,Page_13,Page_14,Page_15,Page_16,Page_17,Page_18,Page_19,Page_20,Page_21,Page_22,Page_23,Page_24,Page_25,Page_26,Page_27,Page_28,Page_29,Page_30,Page_31,Page_32,Page_33,Page_34,Page_35,Page_36,Page_37,Page_38,Page_39,Page_40,Page_41,Page_42,Page_43,Page_44,Page_45,Page_46,Page_47,Page_48,Page_49,Page_50,Page_51,Page_52,Page_53,Page_54,Page_55,Page_56,Page_57,Page_58,Page_59,Page_60,Page_61,Page_62,Page_63,Page_64,Page_65,Page_66,Page_67
0,S5WLXErzUWNQqkDKnRC1j53PD,"[[[1536, 864, 1536, 722, 5083, None, 164841125...","[[[1536, 864, 1536, 722, 1425, None, 164841125...","[[[1536, 864, 1536, 722, 1577, None, 164841125...","[[[1536, 864, 1536, 722, 1607, None, 164841125...","[[[1536, 864, 1536, 722, 1066, None, 164841126...","[[[1536, 864, 1536, 722, 876, None, 1648411261...","[[[1536, 864, 1536, 722, 948, None, 1648411262...","[[[1536, 864, 1536, 722, 1091, None, 164841126...","[[[1536, 864, 1536, 722, 1037, None, 164841126...","[[[1536, 864, 1536, 722, 1230, None, 164841126...","[[[1536, 864, 1536, 722, 732, None, 1648411266...","[[[1536, 864, 1536, 722, 1087, None, 164841127...","[[[1536, 864, 1536, 722, 1350, None, 164841127...","[[[1536, 864, 1536, 722, 1224, None, 164841127...","[[[1536, 864, 1536, 722, 968, None, 1648411274...","[[[1536, 864, 1536, 722, 957, None, 1648411275...","[[[1536, 864, 1536, 722, 1029, None, 164841127...","[[[1536, 864, 1536, 722, 1155, None, 164841127...","[[[1536, 864, 1536, 722, 1105, None, 164841127...","[[[1536, 864, 1536, 722, 1010, None, 164841127...","[[[1536, 864, 1536, 722, 972, None, 1648411280...","[[[1536, 864, 1536, 722, 1020, None, 164841128...","[[[1536, 864, 1536, 722, 1087, None, 164841128...","[[[1536, 864, 1536, 722, 1290, None, 164841128...","[[[1536, 864, 1536, 722, 1607, None, 164841128...","[[[1536, 864, 1536, 722, 981, None, 1648411287...","[[[1536, 864, 1536, 722, 1005, None, 164841128...","[[[1536, 864, 1536, 722, 1277, None, 164841128...","[[[1536, 864, 1536, 722, 1141, None, 164841129...","[[[1536, 864, 1536, 722, 1008, None, 164841129...","[[[1536, 864, 1536, 722, 1029, None, 164841129...","[[[1536, 864, 1536, 722, 1120, None, 164841129...","[[[1536, 864, 1536, 722, 1189, None, 164841129...","[[[1536, 864, 1536, 722, 1175, None, 164841129...","[[[1536, 864, 1536, 722, 978, None, 1648411297...","[[[1536, 864, 1536, 722, 1030, None, 164841129...","[[[1536, 864, 1536, 722, 1021, None, 164841129...","[[[1536, 864, 1536, 722, 1118, None, 164841130...","[[[1536, 864, 1536, 722, 1305, None, 164841130...","[[[1536, 864, 1536, 722, 1083, None, 164841130...","[[[1536, 864, 1536, 722, 1037, None, 164841130...","[[[1536, 864, 1536, 722, 1049, None, 164841130...","[[[1536, 864, 1536, 722, 1049, None, 164841130...","[[[1536, 864, 1536, 722, 1198, None, 164841130...","[[[1536, 864, 1536, 722, 1048, None, 164841130...","[[[1536, 864, 1536, 722, 1003, None, 164841130...","[[[1536, 864, 1536, 722, 966, None, 1648411310...","[[[1536, 864, 1536, 722, 1045, None, 164841131...","[[[1536, 864, 1536, 722, 1118, None, 164841131...","[[[1536, 864, 1536, 722, 989, None, 1648411313...","[[[1536, 864, 1536, 722, 938, None, 1648411314...","[[[1536, 864, 1536, 722, 1097, None, 164841131...","[[[1536, 864, 1536, 722, 1794, None, 164841131...","[[[1536, 864, 1536, 722, 1068, None, 164841131...","[[[1536, 864, 1536, 722, 972, None, 1648411320...","[[[1536, 864, 1536, 722, 1139, None, 164841132...","[[[1536, 864, 1536, 722, 1207, None, 164841132...","[[[1536, 864, 1536, 722, 1220, None, 164841132...","[[[1536, 864, 1536, 722, 993, None, 1648411324...","[[[1536, 864, 1536, 722, 1119, None, 164841132...","[[[1536, 864, 1536, 722, 1090, None, 164841132...","[[[1536, 864, 1536, 722, 1194, None, 164841132...","[[[1536, 864, 1536, 722, 1284, None, 164841132...","[[[1536, 864, 1536, 722, 1085, None, 164841133...","[[[1536, 864, 1536, 722, 1117, None, 164841133...","[[[1536, 864, 1536, 722, 1139, None, 164841133..

Creating dataframe for H sessions.

In [9]:
df_h = create_df(H_FILES, h_from_excel, pairing_h)
df_h[:5]

C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\117512531.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\117512531.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\117512531.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\117512531.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users

Finished 'create_df' in 8.7730 secs


,Session,Page_0,Page_1,Page_2,Page_3,Page_4,Page_5,Page_6,Page_7,Page_8,Page_9,Page_10,Page_11,Page_12,Page_13,Page_14,Page_15,Page_16,Page_17,Page_18,Page_19,Page_20,Page_21,Page_22,Page_23,Page_24,Page_25,Page_26,Page_27,Page_28,Page_29,Page_30,Page_31,Page_32,Page_33,Page_34,Page_35,Page_36,Page_37,Page_38,Page_39,Page_40,Page_41,Page_42,Page_43,Page_44,Page_45,Page_46,Page_47,Page_48,Page_49,Page_50,Page_51,Page_52,Page_53,Page_54,Page_55,Page_56,Page_57,Page_58,Page_59,Page_60,Page_61,Page_62,Page_63,Page_64,Page_65,Page_66,Page_67
0,sMjVKK5bH9bGGEeDF4TMra2xb,"[[[1536, 864, 1536, 722, 16908, [{'start': 839...","[[[1536, 864, 1536, 722, 1560, None, 164841112...","[[[1536, 864, 1536, 722, 1052, None, 164841112...","[[[1536, 864, 1536, 722, 1883, None, 164841112...","[[[1536, 864, 1536, 722, 1019, None, 164841112...","[[[1536, 864, 1536, 722, 1025, None, 164841113...","[[[1536, 864, 1536, 722, 1171, None, 164841113...","[[[1536, 864, 1536, 722, 1280, None, 164841113...","[[[1536, 864, 1536, 722, 1490, None, 164841113...","[[[1536, 864, 1536, 722, 1227, None, 164841113...","[[[1536, 864, 1536, 722, 1149, None, 164841113...","[[[1536, 864, 1536, 722, 1086, None, 164841113...","[[[1536, 864, 1536, 722, 1065, None, 164841113...","[[[1536, 864, 1536, 722, 536, None, 1648411140...","[[[1536, 864, 1536, 722, 1709, None, 164841114...","[[[1536, 864, 1536, 722, 1120, None, 164841114...","[[[1536, 864, 1536, 722, 1048, None, 164841114...","[[[1536, 864, 1536, 722, 1185, None, 164841114...","[[[1536, 864, 1536, 722, 1865, None, 164841114...","[[[1536, 864, 1536, 722, 1133, None, 164841115...","[[[1536, 864, 1536, 722, 1125, None, 164841115...","[[[1536, 864, 1536, 722, 1096, None, 164841115...","[[[1536, 864, 1536, 722, 1291, None, 164841115...","[[[1536, 864, 1536, 722, 1568, None, 164841115...","[[[1536, 864, 1536, 722, 1120, None, 164841115...","[[[1536, 864, 1536, 722, 1062, None, 164841115...","[[[1536, 864, 1536, 722, 1027, None, 164841115...","[[[1536, 864, 1536, 722, 1081, None, 164841116...","[[[1536, 864, 1536, 722, 1115, None, 164841116...","[[[1536, 864, 1536, 722, 1194, None, 164841116...","[[[1536, 864, 1536, 722, 1112, None, 164841116...","[[[1536, 864, 1536, 722, 1366, None, 164841116...","[[[1536, 864, 1536, 722, 1258, None, 164841116...","[[[1536, 864, 1536, 722, 1114, None, 164841116...","[[[1536, 864, 1536, 722, 1625, None, 164841116...","[[[1536, 864, 1536, 722, 1047, None, 164841117...","[[[1536, 864, 1536, 722, 1138, None, 164841117...","[[[1536, 864, 1536, 722, 1116, None, 164841117...","[[[1536, 864, 1536, 722, 1116, None, 164841117...","[[[1536, 864, 1536, 722, 1109, None, 164841117...","[[[1536, 864, 1536, 722, 1229, None, 164841117...","[[[1536, 864, 1536, 722, 1122, None, 164841117...","[[[1536, 864, 1536, 722, 1309, None, 164841117...","[[[1536, 864, 1536, 722, 1161, None, 164841117...","[[[1536, 864, 1536, 722, 1367, None, 164841118...","[[[1536, 864, 1536, 722, 1062, None, 164841118...","[[[1536, 864, 1536, 722, 1144, None, 164841118...","[[[1536, 864, 1536, 722, 1230, None, 164841118...","[[[1536, 864, 1536, 722, 1204, None, 164841118...","[[[1536, 864, 1536, 722, 1106, None, 164841118...","[[[1536, 864, 1536, 722, 1185, None, 164841118...","[[[1536, 864, 1536, 722, 1723, None, 164841118...","[[[1536, 864, 1536, 722, 2991, None, 164841119...","[[[1536, 864, 1536, 722, 1742, None, 164841119...","[[[1536, 864, 1536, 722, 1008, None, 164841119...","[[[1536, 864, 1536, 722, 1063, None, 164841119...","[[[1536, 864, 1536, 722, 1093, None, 164841119...","[[[1536, 864, 1536, 722, 1389, None, 164841119...","[[[1536, 864, 1536, 722, 1134, None, 164841120...","[[[1536, 864, 1536, 722, 1305, None, 164841120...","[[[1536, 864, 1536, 722, 1236, None, 164841120...","[[[1536, 864, 1536, 722, 1158, None, 164841120...","[[[1536, 864, 1536, 722, 1217, None, 164841120...","[[[1536, 864, 1536, 722, 1486, None, 164841120...","[[[1536, 864, 1536, 722, 1586, None, 164841120...","[[[1536, 864, 1536, 722, 1890, None, 164841121..

Appending 2 created dataframes to 1.

In [10]:
df_all = df_fg.append(df_h, ignore_index=True)
result = df_all.to_json(orient="records")

C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_28400\2262360144.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_fg.append(df_h, ignore_index=True)


Saving created dataframe to JSON.

In [11]:
with open(SESSION_EVENTS_FROM_BAKED_JSON, 'w') as outfile:
    outfile.write(result)